# Electronic Safe

In [ ]:
import Adafruit_DHT, time, pyfirmata, chestnut.arduino, RPi.GPIO as gpio
from subprocess import Popen, PIPE
from pyfirmata import Arduino
from time import sleep
from gpiozero import Servo
from ifttt_webhook import IftttWebhook

######################################### Setup #########################################################
# Initializing the board variable
board = None

# Function to interface with Arduino and Raspberry Pi
def interface_arduino():
    global board
    # Get the port where the Arduino is connected
    stdout = Popen('dmesg | grep -v disconnect | grep -Eo "tty(ACM|USB)." | tail -1', shell=True, stdout=PIPE).stdout
    port = str(stdout.read(), 'utf-8')
    
    # Converting the port to string and removing the new line character
    port_str = '/dev/' + port
    port = port_str.split('\n')[0]
    
    # Flashing Firmata to the Arduino board
    chestnut.arduino.flash_firmata(board='uno', port= port, debug=True) # port from Linux
    board = Arduino(port)

# Call the function to interface with Arduino
interface_arduino()

######################################### Global Definitions ###############################################
# Default key
key_default       = ['1','1','1','1'] # default key
key               = [] # current key
key_code          = [] # stored key

# Pin mapping for the columns and rows of the keypad
column            = [12, 16, 20, 21]
row               = [18, 23, 24, 25]

# Pin mappings for Arduino
in_pir            = board.digital[9] # PIR sensor input
out_buzzer        = board.digital[2] # buzzer output
out_led_red       = board.digital[4] # red LED output
out_led_green     = board.digital[5] # green LED output
out_led_blue      = board.digital[6] # blue LED output
servo             = None # Servo motor for opening the safe


# Status flags
is_keypad_running = False  # Flag to keep track if the keypad is running
is_motion         = False  # Flag to keep track if motion is detected
is_unlocked       = False  # Flag to keep track if the safe is unlocked
temperature       = 0.0  # Initial temperature
humidity          = 0.0  # Initial humidity

# Default values for temperature and humidity
temperature_default   = 50
humidity_default      = 200

# Threshold values for temperature and humidity
temperature_threshold = temperature_default
humidity_threshold    = humidity_default
ifttt =  IftttWebhook('cmtJ2kEch4ka-UYyvORA3X') # IFTTT webhook
matrix            = [["1","2","3","A"],
                     ["4","5","6","B"],
                     ["7","8","9","C"],
                     ["*","0","#","D"]]

######################################### IFTTT Webhooks API calls #######################################
# Logs the data into an excel sheet periodically or for every emergency
def ifttt_Log_Data(is_SOS):
    global is_unlocked
    global temperature
    global humidity
    global is_motion
    global temperature_threshold
    global humidity_threshold
    global key_code
    global is_keypad_running
    global ifttt
    
    # Log the data with lock status, temperature, and humidity
    col2 = 'Unlocked' if is_unlocked else 'Locked'
    col3 = 'Temperatue={0:0.1f}°C, Max Limit={1:0.1f}°C'.format (temperature, temperature_threshold)
    col4 = 'Humidity={0:0.1f}, Max Limit={1:0.1f}'.format (humidity, humidity_threshold)
    if is_motion == True and not is_unlocked: col2 = 'WARNING: Motion detected inside the locked safe!'
    
    # Log the data for every emergency
    ifttt.trigger('Electronic safe Log', col2, col3, col4)
    
    if is_SOS:
        ifttt.trigger('Electronic safe SOS', col2, col3, col4)
        print('# Email sent to the registered email-address')

# Check the keypad matrix to get the pressed key value
def keypress_check():
    # Loop through the columns of the keypad
    for x in range(10):
        for j in range(4):
            # Set the current column to low
            gpio.output(column[j], 0)
            # Loop through the rows of the keypad
            for i in range(4):
                # Check for low input signal on the current row
                if gpio.input(row[i]) == 0:
                    # Store the corresponding key value in a variable
                    user_in = matrix[i] [j]
                    sleep(0.2)
                    # Wait for the key to be released
                    while gpio.input (row[i]) == 0:
                        pass
                    return user_in
            # Set the current column to high
            gpio.output(column[j], 1)
    # Return False if no key press is detected
    return False

# Function to setup the password
def setupKeypad(is_firstTime):
    global key
    key = []
    while len(key) != 4:
        in_custom = '*'
        
        # Check if it's the first time setup
        if is_firstTime:
            print('Do you want to setup a new password? \n \t- Press "*" for new\n\t- Press "#" for default')
            
            in_custom = False
            # Continuously check for user input
            while in_custom == False:
                in_custom = keypress_check()
        in_string = []
        
        # If user chooses to set up a new password
        if(in_custom == '*'):
            print('Enter a new password: ', end  = ' ')
            # Continuously get user input for password
            while(len(in_string) < 4):
                key_pressed = keypress_check()
                if(len(in_string) < 4) and (key_pressed != False):
                    in_string.append(key_pressed)
                    print(key_pressed, end  = ' ')
        
        # If user chooses to use the default password
        elif(in_custom == '#'):
            in_string = '1111'
        
        # If user inputs an invalid option
        else:
            print('# Error: Please enter a proper input!')
        
        # Check if the input is a valid password (4 digits)
        if(not ''.join(in_string).isdigit()):
            print('\n# Error: Please enter a proper password!')
        else:
            # Store the password in the `key` variable
            for letter in in_string:
                key.append(letter)
            print ('\n# Password saved: ',' '.join(key))

# Get the pressed key and verify the password
def keypad_check():
    global key
    global key_code
    global is_keypad_running
    global is_unlocked
    global servo
    
    # When no key is pressed, print the welcome message
    if(len(key_code) == 0) and (not is_keypad_running):
        print("Enter the password: ", end  = ' ')
        is_keypad_running = True

    # Get the key that is pressed on the keypad
    key_pressed = keypress_check()
    
    # If a key is pressed and the password is not complete
    if(len(key_code) < 4) and (key_pressed != False):
        # Add the key to the password code
        key_code.append(key_pressed)
        print(key_pressed, end  = ' ')
        
    # If the password is complete
    elif(len(key_code) == 4):
        if (key_code ==  key):
            print('\n******* Access Granted! *******')
            # Turn on and off the green LED and buzzer to indicate access granted
            out_buzzer.write(1)
            out_led_green.write(1)
            # Move the servo to the open position
            servo.mid()

            sleep(0.2)
            out_buzzer.write(0)
            out_led_green.write(0)

            # Set the flag to indicate that the system is unlocked
            is_unlocked = True
            
        # If the entered password is incorrect
        else:
            print('\n******* Wrong Password! *******')
            # Turn on and off the red LED and buzzer to indicate wrong password
            out_buzzer.write(1)
            out_led_red.write(1)

            sleep(1)
            out_buzzer.write(0)
            out_led_red.write(0)
            # Set the flag to indicate that the system is locked
            is_unlocked = False
        # Reset the keypad running flag and the password code
        is_keypad_running = False
        key_code = []

# Activation of LED and buzzer to simulate warning signal
def warning_signal():
    global out_buzzer
    global out_led_red
    
    out_buzzer.write(1)
    out_led_red.write(1)

    sleep(0.5)
    out_buzzer.write(0)
    out_led_red.write(0)
    sleep(0.5)
    
# Read DHT11 Sensor information and invoke the warning signal
def read_dht11():
    global temperature
    global humidity
    global temperature_threshold
    global humidity_threshold
    
    humidity, temperature = Adafruit_DHT.read_retry(Adafruit_DHT.DHT11, 17)

    if humidity is not None and temperature is not None:
        if(humidity > humidity_threshold or temperature > temperature_threshold):
            print('\n# WARNING: Temperatue={0:0.1f}°C, Humidity={1:0.1f}'.format (temperature, humidity))
            print('Press:\t"D" to silence the alarm.')
            is_keypad_running = False
            key_code = []
            ifttt_Log_Data(True)
            key_pressed = False
            while(humidity > humidity_threshold or temperature > temperature_threshold) and (key_pressed != 'D'):
                humidity, temperature = Adafruit_DHT.read_retry(Adafruit_DHT.DHT11, 17)
                warning_signal()

                key_pressed = keypress_check()

def read_motion():
    global key_code
    global is_keypad_running
    global is_unlocked
    global is_motion
    
    # Check if any motion detected
    is_motion = in_pir.read()
    if is_motion == True and not is_unlocked:
        print("\n# WARNING: Motion detected inside the safe!")
        is_keypad_running = False
        key_code = []
        ifttt_Log_Data(True)
        
        while is_motion == True:
            is_motion = in_pir.read()
            warning_signal()

# Get new limits from user for humidity and temperature monitoring
def get_limit_data():
    key_pressed = False
    value = []
    while key_pressed != '#':
        key_pressed = False
        while key_pressed == False:
            key_pressed = keypress_check()
        
        print(key_pressed, end  = ' ')
        if key_pressed.isdigit():
            value.append(key_pressed)
        elif key_pressed != '#':
            value = []
            print('# Error: Please enter a proper input!')
            print('Enter value (press "#" to complete): ', end = ' ')
            
    return int(''.join(value))

# Set new limits for the humidity and temperature monitoring
def set_new_limits():
    global temperature_threshold
    global humidity_threshold
    
    print('Enter Temperature Max Limit (press "#" to complete): ', end = ' ')
    temperature_threshold = get_limit_data()
    
    print('\nEnter Humidity Max Limit (press "#" to complete): ', end = ' ')
    humidity_threshold = get_limit_data()
    print('\n')
    
def clear_outpins():
    # Clear all the out pins
    out_buzzer.write(0)
    out_led_red.write(0)
    out_led_green.write(0)
    out_led_blue.write(0)
    
    # Clear the keypad pins
    for i in range(4):
        gpio.setup(column[i], gpio.OUT)
        gpio.output(column[i], 1)
        gpio.setup(row[i], gpio.IN, pull_up_down = gpio.PUD_UP)
    
# Initialize all the pins for Arduino and Raspberry pie, and initialize all the global variables
def init():
    global key
    global key_code
    global is_keypad_running
    global is_unlocked
    global is_motion
    global temperature
    global humidity
    global servo
    
    # Initialize the global variables
    is_unlocked        = False
    is_keypad_running  = False
    is_motion          = False
    temperature        = None
    humidity           = None
    key_code           = []
    key                = []
    
    # Pin mappings for Raspberry pie
    gpio.setwarnings(False)
    gpio.setmode(gpio.BCM)
    gpio.setup(23, gpio.IN)
    gpio.setup(12, gpio.OUT)

    # Pin mappings for Arduino
    in_pir        = board.digital[9]
    out_buzzer    = board.digital[2]
    out_led_red   = board.digital[4]
    out_led_green = board.digital[5]
    out_led_blue  = board.digital[6]
    servo = Servo(26, min_pulse_width=0.0006, max_pulse_width=0.0026)
    servo.min()

    pyfirmata.util.Iterator(board).start()
    in_pir.mode   = pyfirmata.INPUT
    
    # Clear the output pins
    clear_outpins()
    read_dht11()
    setupKeypad(True)

def inf_loop():
    global servo
    global is_unlocked
    global temperature
    global humidity
    global temperature_threshold
    global humidity_threshold
    
    # Start the timer to optimize the monitoring time in discreetly manner
    start_dht11 = time.time()
    start_log = time.time()
    while True:
        # Calculate the elapsed time
        end = time.time()
        elapsed_time_log = int(end - start_log)
        elapsed_time_dht11 = int(end - start_dht11)
        
        # Check if safe is locked or unlocked
        if not is_unlocked:
            # If safe is locked then wait for the password while monitoring the sensor conditions
            keypad_check()
        else:
            # If safe is unlocked then allow the user to access and modify certain elements of the safe
            print('# Options:\tA: Lock the safe')
            print('# \t\tB: Change the sensor thresholds')
            print('# \t\tC: Send current status in an email')
            print('# \t\tD: Set a new password')
            print('# \t\t#: View the sensor data')
            print('# \t\t*: Reset all')
            
            # Provide the access menu interactions to the user
            key_pressed = False
            while key_pressed == False:
                key_pressed = keypress_check()
            
            #### Navigate the access menu based on the key press ####
            # Lock the safe manually
            if key_pressed == 'A':
                is_unlocked = False
                servo.min()
                print('# Locked!')
            
            # Modify the temperature and humidity sensor warning levels
            elif key_pressed == 'B':
                set_new_limits()
            
            # Log the current status of the hardware to the cloud
            elif key_pressed == 'C':
                ifttt_Log_Data(True)
            
            # Allow user to set a new password when safe is unlocked
            elif key_pressed == 'D':
                setupKeypad(False)
            
            # Print the sensor data
            elif key_pressed == '#':
                print('\n# Temperatue={0:0.1f}°C, Max Limit={1:0.1f}°C'.format (temperature, temperature_threshold))
                print('# Humidity={0:0.1f}, Max Limit={1:0.1f}\n'.format (humidity, humidity_threshold))
                sleep(1)
            
            # Reset all the elements to defaults
            elif key_pressed == '*':
                key = key_default
                temperature_threshold = temperature_default
                humidity_threshold    = humidity_default
                print ('\n# Reset all complete!')
                print ('# \tPassword saved: ',' '.join(key))
                print ('# \tTemperature threshold: ',temperature_threshold)
                print ('# \tHumidity threshold: ',humidity_threshold,'\n')
                sleep(1)
        
        # Clear previous sensor pins before reading the data again
        clear_outpins()
        
        # Check for motion inside the safe
        read_motion()
        
        # Every 10 seconds, read the temperature and humidity data
        if(int(end - start_dht11) > 10):
            read_dht11()
            start_dht11 = time.time()
            
        # Every 10 minutes, log the status of the safe to the google sheets
        if(int(end - start_log) > 600):
            ifttt_Log_Data(False)
            start_log = time.time()

try:
    # Initialize all of the modules and set the global variables
    init()
    # Infinite loop for continous running and monitoring of the hardware
    inf_loop()

# For any interruptions Safely clearing the pins before exiting the code
except KeyboardInterrupt:
#except Exception as e:
    clear_outpins()
    servo.close()
    print("\n# Program stopped")
    gpio.cleanup()
   
# EOF